In [1]:
! python3 -m ipykernel install --user --name=main-venv

Installed kernelspec main-venv in /home/roman/.local/share/jupyter/kernels/main-venv


In [2]:
import os

import json

from tqdm import tqdm

import pandas as pd
import numpy as np

from pydantic import BaseModel, Field

from typing import List

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.exceptions import OutputParserException

from langchain_gigachat.chat_models import GigaChat

from datasets.parser_dataset import ParserDataset

In [3]:
DATA_DIR = "data"

# ✅Build Giga solution

In [4]:
class GigaOutput(BaseModel):

    topic: str = Field(description='''
        Название темы на форуме.
    ''')
    menace_status: str = Field(description='''
        Ответ от Гигачата на вопрос, обсуждается ли
        в этой теме нарушение закона с целью личной выгоды.
    '''),
    comment: str = Field(description='''
        Небольшой комментарий от Гигачата,
        почему он посчитал обсуждение в теме мошенническими.
    ''')
    highlights: str = Field(description='''
        Главные, по мнению гигачата, слова. 
    ''')

class GigaMenace:

    def __init__(self, token: str, prompt: str, model: str = 'GigaChat-2'):

        self.model = GigaChat(
            credentials=token,
            verify_ssl_certs=False,
            model=model
        ).with_structured_output(GigaOutput, method='json_mode')

        self.prompt = prompt
        self.messages = []

    def _init_chat(self, messages: List[str]):
        
        self.messages = [
            SystemMessage(content=self.prompt)
        ]

        for message in messages:
            self.messages.append(
                HumanMessage(content=str(message))
            )
    
    def inference(self, topics: List[str]):
        
        res = []
        
        for topic in tqdm(topics):

            try:
                self._init_chat(topics[topic])
                answer = self.model.invoke(
                    self.messages
                )
            except OutputParserException:
                answer = {'topic': topic, 'menace_status': 'да', 'comment': 'чувствительная тема', 'highlights': ''}
                
            res.append(dict(answer))
        
        return res

In [5]:
with open('gc-token.txt', 'r') as file:
    TOKEN = file.read()

system_prompt = '''
    Ты - помошник по борьбе с онлайн мошенничеством в банке.
    Тебе дают диалог на определенную тему на одном из онлайн банковских форумов.
    Твоя задача - определить, содержит ли диалог элементы мошенничества - 
    если да, ответь "да", иначе - "нет".
    Также дай небольшой комментарий, почему ты так решил.
    Главные слова в диалоге - подсвети.

    Проанализируй тему и верни ответ в формате JSON со структурой:
        {
          "topic": "оригинальное название темы"
          "menace_status": "да/нет",
          "comment": "твой короткомментарий",
          "highlights": "самые яркие тезисы диалога, если мошенничество, то явные слова о мошенничестве"
        }

    Запрещено:
        - Добавлять посторонний текст
        - Менять структуру JSON
'''

giga = GigaMenace(TOKEN, system_prompt)

# ✅Detect fraud in Kupus.ru

In [8]:
kupus_topics_df = pd.read_excel(os.path.join(DATA_DIR, 'collected/kupus_headers.xlsx'))
kupus_topics_mapped = pd.read_csv(os.path.join(DATA_DIR, 'kupus_mapping_credits.csv'))
kupus_topics_mapped = kupus_topics_mapped[kupus_topics_mapped['menace_status'] == 'да']

kupus_topics_df = kupus_topics_df.merge(kupus_topics_mapped, left_on='flud name', right_on='topic')

kupus_comments = pd.read_excel(os.path.join(DATA_DIR, 'collected/kupus_comments.xlsx'))

In [21]:
messages = dict()

for topic in set(kupus_topics_df['topic name'].values):
    dialog = kupus_comments[kupus_comments['topic name'] == topic]['message'].values
    messages[topic] = dialog

kupus_res = giga.inference(messages)

100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.02it/s]


In [22]:
kupus_res

[{'topic': 'Секреты и Лайфхаки',
  'menace_status': 'да',
  'comment': 'чувствительная тема',
  'highlights': ''},
 {'topic': 'Мошеннические транзакции',
  'menace_status': 'да',
  'comment': 'чувствительная тема',
  'highlights': ''}]

# ✅Detect fraud hranidengi

In [23]:
hranidengi_topics_df = pd.read_excel(os.path.join(DATA_DIR, 'collected/hranidengi_headers.xlsx'))
hranidengi_topics_mapped = pd.read_csv(os.path.join(DATA_DIR, 'hranidengi_mapping_credits.csv'))
hranidengi_topics_mapped = hranidengi_topics_mapped[hranidengi_topics_mapped['menace_status'] == 'да']

hranidengi_topics_df = hranidengi_topics_df.merge(hranidengi_topics_mapped, left_on='flud name', right_on='topic')

hranidengi_comments = pd.read_excel(os.path.join(DATA_DIR, 'collected/hranidengi_comments.xlsx'))

Idea - same, but open 